In [2]:
import pandas as pd
import torch

In [91]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',500)  

In [123]:
# Load datasets
file_path_input = "data/ABS-MOPS Variables - December 11 2024.xlsm"
file_path_mdr = "data/mdr Variables 1.xlsx"

input_df = pd.read_excel(file_path_input, sheet_name="Data Sheet", header=12).rename(columns={'Unnamed: 3':'Legacy Variable'})
mdr_df = pd.read_excel(file_path_mdr)

input_columns_to_check = ["Legacy Variable", "Variable Name *", "Description *"]
mdr_columns_to_check = ["name", "definition"]

C:\Users\cyy12\mdr\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [125]:
# input_df[input_columns_to_check].head()

In [127]:
# mdr_df[mdr_columns_to_check].head()

In [129]:
# Remove rows with any null values and remove duplicates
print(input_df.shape)
print(mdr_df.shape)
input_df = input_df.dropna(subset=input_columns_to_check)
mdr_df = mdr_df.dropna(subset=mdr_columns_to_check)
print(input_df.shape)
print(mdr_df.shape)
input_df = input_df.drop_duplicates(subset=["Variable Name *"])
mdr_df = mdr_df.drop_duplicates(subset=["name"])
print(input_df.shape)
print(mdr_df.shape)

(395, 28)
(41626, 9)
(392, 28)
(41451, 9)
(371, 28)
(37836, 9)


In [131]:
# Remove place holders like TBD

placeholder_vars = ['tbd'] # lower case

for var in placeholder_vars:
    for col in mdr_columns_to_check:
        to_drop = mdr_df[mdr_df[col].str.lower()==var].index
        mdr_df = mdr_df.drop(to_drop)

for var in placeholder_vars:
    for col in input_columns_to_check:
        to_drop = input_df[input_df[col].str.lower()==var].index
        input_df = input_df.drop(to_drop)

In [133]:
# double check null 
missing_columns = input_df[input_columns_to_check].isnull().any()
print('if still missing values in input df:\n',missing_columns)

missing_columns = mdr_df[mdr_columns_to_check].isnull().any()
print('if still missing values in mdr df:\n',missing_columns)

if still missing values in input df:
 Legacy Variable    False
Variable Name *    False
Description *      False
dtype: bool
if still missing values in mdr df:
 name          False
definition    False
dtype: bool


In [135]:
# double check placeholders:
for var in placeholder_vars:
    print(f'placeholder value: {var}')
    
    df_i = input_df[input_columns_to_check].map(lambda x: x.lower() if isinstance(x, str) else x)
    value_exists = (df_i == var).any().any()
    print(f'if any {var} in input df: \n',value_exists)
    
    df_m = mdr_df[mdr_columns_to_check].map(lambda x: x.lower() if isinstance(x, str) else x)
    value_exists = (df_m == var).any().any()
    print(f'if any {var} in mdr df: \n',value_exists)

placeholder value: tbd
if any tbd in input df: 
 False
if any tbd in mdr df: 
 False


In [137]:
print(input_df.shape)
print(mdr_df.shape)

(371, 28)
(36618, 9)


In [139]:
# Save cleaned data to new files
input_df.to_excel("cleanInput2.xlsx", index=False)
mdr_df.to_excel("cleanMDR2.xlsx", index=False)